In [1]:
from statistics import ClassHistory
import fake_data as fd

mmsr_data = fd.mmsr_data
maintenance_periods_list = fd.maintenance_periods_list
deposit_rate_data = fd.deposit_rate_data


history = ClassHistory()
history.build_from_data(mmsr_data, deposit_rate_data, maintenance_periods_list)


c:\Users\VictorLeCoz\OneDrive - Quant Decisions S.L\Documents\PhD\secured-inter-bank-network\fake_data.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deposit_rate_data.new_deposit_rate.loc[


0 incorect lines in the data base
